In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

parent_dir = os.path.join(os.getcwd(), '..')
if not parent_dir in sys.path:
    sys.path.insert(0, parent_dir)
import mlutil 
pp, fd = mlutil.pp, mlutil.fd

# Precision
pd.set_option('precision', 2) 
np.set_printoptions(precision=2)

# Warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)